<a href="https://colab.research.google.com/github/Usama-zia/Project-2/blob/main/project_2_Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Installing Libraries and Dependencies


In [ ]:

#Installing libraries and Dependencies
!pip install sentencepiece
!pip install transformers
from transformers import RobertaTokenizer,TFRobertaForSequenceClassification
from transformers import TFTrainer, TFTrainingArguments
from transformers import XLMTokenizer, TFXLMForSequenceClassification
import torch
import tensorflow as tf
import numpy as np
import seaborn as sns
import pandas as pd
import csv
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
import string
import math
import spacy
from sklearn.feature_extraction.text import CountVectorizer
from wordcloud import WordCloud
from textwrap import wrap
from textblob import TextBlob

#Cloning the Dataset from Github Repository(https://github.com/Usama-zia/Project-2)

In [ ]:
# remove dataset directory if already exists
!rm -rf project-2

# fetch dataset
!git clone https://github.com/Usama-zia/Project-2.git

fatal: destination path 'Project-2' already exists and is not an empty directory.


#Function for preprocessing Datasets

In [ ]:

def preprocessing_data(data):
  print('============Before Preprocessing=============')

  for index,text in enumerate(data[0][5:10]):
    print('tweet %d:\n'%(index+1),text)
  #removing punctuations
  data[0]=data[0].apply(lambda x: re.sub('[%s]' % re.escape(string.punctuation), '', x))
  #removing extra spaces
  data[0]=data[0].apply(lambda x: re.sub(' +',' ',x))
  #removing @user
  data[0] = data[0].str.strip('user')

  #after Preprocessing
  print('============After Preprocessing=============')
  for index,text in enumerate(data[0][0:5]):
    print('tweet %d:\n'%(index+1),text)
  return data

#Some Constant Used for Loading the Dataset

In [ ]:
#folders = [emotion, hate ,sentiment]
emotion = '/content/Project-2/datasets/emotion'
hate = '/content/Project-2/datasets/hate'
sentiment = '/content/Project-2/datasets/sentiment'
train_text = 'train_text.txt'
train_labels = 'train_labels.txt'
test_text = 'test_text.txt'
test_labels = 'test_labels.txt'
val_text = 'val_text.txt'
val_labels = 'val_labels.txt'
maps = 'mapping.txt'

#Function for Loading Datasets

In [ ]:
#load and preporcess train and val dataset for emotion
emotion = '/content/Project-2/datasets/emotion'
data_set=emotion
with open(os.path.join(data_set, train_text)) as f:
  train_texts = pd.read_csv(f, sep="\n", header=None)
  with open(os.path.join(data_set,train_labels)) as f:
    train_labels = pd.read_csv(f, sep="\n", header=None)

with open(os.path.join(data_set, val_text)) as f:
  val_texts = pd.read_csv(f, sep="\n", header=None)
  with open(os.path.join(data_set, val_labels)) as f:
    val_labels = pd.read_csv(f, sep="\n", header=None)

with open(os.path.join(data_set, test_text)) as f:
  test_texts = pd.read_csv(f, sep="\n", header=None)
  with open(os.path.join(data_set, test_labels)) as f:
    test_labels = pd.read_csv(f, sep="\n", header=None)

train_data = preprocessing_data(train_texts)
val_data = preprocessing_data(val_texts)
test_data = preprocessing_data(test_texts)



In [ ]:
train_d=train_data[0].tolist()
val_d=val_data[0].tolist()
test_d=test_data[0].tolist()

In [ ]:
tokenizer =XLMTokenizer.from_pretrained('xlm-mlm-en-2048')
train_encodings = tokenizer(train_d, truncation=True, padding=True)
val_encodings = tokenizer(val_d, truncation=True, padding=True)
test_encodings = tokenizer(test_d, truncation=True, padding=True)


In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices((dict(train_encodings),train_labels))
val_dataset = tf.data.Dataset.from_tensor_slices((dict(val_encodings),val_labels))
test_dataset = tf.data.Dataset.from_tensor_slices((dict(test_encodings),test_labels))

In [ ]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='macro')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

In [ ]:

training_args = TFTrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=3,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
)

with training_args.strategy.scope():
    model =TFXLMForSequenceClassification.from_pretrained('xlm-mlm-en-2048')

trainer = TFTrainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset             # evaluation dataset
)

trainer.train()

In [ ]:
#evaluation
trainer.evaluate()


In [ ]:
#testing
tt=trainer.predict(test_dataset)

In [ ]:
#socres
print(tt.metrics)